### Pipelines

<include type="image">

###### This is not a pipe
    
![](../img/cecinestpas.jpg)

</include>


So far, we've established a general workflow where we:


1. **Clean the data Fill/impute/drop `NaN` values**
   - One-hot encode categorical variables
   - Label-encode target if categorical
   - Check for skew / deskew

1. **Preprocess the data**
   - Feature selection (`SelectKBest, SelectFromModel, SelectPercentile, RFE`, etc.)
   - Scaling (`StandardScaler, MinMaxScaler`)

1. **Modeling**
   - Classification (`KNeighborsClassifier, LogisticRegression`, etc.)
   - Regression (`Lasso, Ridge, ElasticNet`, etc.)
  
For every dataset, we've done some version of all of these. Pipelines give us a convenient way to chain these tasks together. As a result, we can feed cleaned data into a pipeline and a trained model a the end!

In [1]:
###### Import the Python Numerical Stack

import pandas as pd
import numpy as np

In [2]:
###### Load bike sharing data

bike_data = pd.read_csv('data/bike_sharing.csv',index_col=0)

In [3]:
###### Display a few columns from the head of `DataFrame`

bike_data[['datetime', 'season', 'holiday', 'workingday', 'temp']].head()

,datetime,season,holiday,workingday,temp
0,2011-01-01 00:00:00,1,0,0,9.84
1,2011-01-01 01:00:00,1,0,0,9.02
2,2011-01-01 02:00:00,1,0,0,9.02
3,2011-01-01 03:00:00,1,0,0,9.84
4,2011-01-01 04:00:00,1,0,0,9.84


#### Clean the data

In [4]:
###### Convert the datetime colum to datetime using pd.to_datetime()

bike_data['datetime'] = pd.to_datetime(bike_data['datetime'])

In [5]:
###### Make a feature for the day of week

bike_data['dayofweek'] = bike_data['datetime'].apply(lambda x: x.dayofweek)

In [6]:
###### Make a feature for month

bike_data['month'] = bike_data['datetime'].apply(lambda x: x.month)

In [7]:
###### Make a feature for hour

bike_data['hour'] = bike_data['datetime'].apply(lambda x: x.hour)

In [8]:
###### Drop the datetime column

bike_data.drop('datetime', axis=1, inplace=True)

In [9]:
###### Split up our features and target into features and target

features = bike_data.drop('count',axis=1)

target = bike_data['count']

In [10]:
###### Get dummies of categorical columns

num_cols = ['temp','atemp','humidity','windspeed']
cat_cols = [i for i in features.columns if i not in num_cols]

features_dummies = pd.get_dummies(features, columns=cat_cols)

#### `Pipeline`

`Pipeline` is a class in `sklearn` that allows us to chain steps together. 

We add steps to the pipeline using a list of tuples of the form `[('step name', sklearn object)...]`

Let's make a `Pipeline` that scales the data and fits a `RandomForestRegressor` model.

In [11]:
###### Load necessary classes and functions from Scikit-Learn 

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import (StandardScaler, 
                                   MinMaxScaler)
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split

In [12]:
###### Train-test split your data

split_data = train_test_split(
    features_dummies, 
    target, 
    random_state = 42
)

X_train, X_test, y_train, y_test = split_data

In [13]:
###### Instantiate your pipeline

simple_pipe = Pipeline([
    ('scaler',StandardScaler()), 
    ('lasso', Lasso())
])

In [14]:
###### Fit the pipeline to your training features and target

simple_pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lasso', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))])

In [15]:
###### What's your train r2 score?

simple_pipe.score(X_train, y_train)

0.6388499611022919

In [16]:
###### What's your test r2 score?

simple_pipe.score(X_test, y_test)

0.6274260376321519

We now have a fit `Pipeline` object that scores just like any other model. This consists of a `StandardScaler` and a `Lasso`. What properties does this `Pipeline` have?

- `.steps` gives you a list of tuples containing the names of your steps and the fit object of the step itself.
- `.named_steps` gives you a dictionary with your pipeline objects where the keys are the names and the values are the fit sklearn object.

In [17]:
###### Look at the steps

simple_pipe.steps

[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('lasso', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False))]

In [18]:
###### Look at the named steps

simple_pipe.named_steps

{'scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'lasso': Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False)}

We can access each step and use it if we'd like. Let's look at the mean and standard deviation of our data from the scaler object.

In [19]:
###### Display means from `StandardScaler`

simple_pipe.steps[0][1].mean_

array([2.02449804e+01, 2.36701605e+01, 6.18351298e+01, 1.27913341e+01,
       2.45835375e-01, 2.51224890e-01, 2.49755022e-01, 2.53184713e-01,
       9.70602646e-01, 2.93973542e-02, 3.16756492e-01, 6.83243508e-01,
       6.62175404e-01, 2.60289074e-01, 7.74130328e-02, 1.22488976e-04,
       1.41474767e-01, 1.42822146e-01, 1.43312102e-01, 1.43434591e-01,
       1.41597256e-01, 1.44169525e-01, 1.43189613e-01, 7.96178344e-02,
       8.30475257e-02, 8.31700147e-02, 8.14551690e-02, 8.56197942e-02,
       8.41499265e-02, 8.31700147e-02, 8.47623714e-02, 8.18226360e-02,
       8.51298383e-02, 8.51298383e-02, 8.29250367e-02, 4.28711416e-02,
       4.10338070e-02, 4.22586967e-02, 4.00538951e-02, 4.12787849e-02,
       4.20137188e-02, 4.16462518e-02, 4.12787849e-02, 4.21362077e-02,
       4.12787849e-02, 4.18912298e-02, 4.06663400e-02, 4.11562959e-02,
       4.04213621e-02, 4.10338070e-02, 4.22586967e-02, 4.18912298e-02,
       4.31161195e-02, 4.37285644e-02, 4.17687408e-02, 4.15237629e-02,
      

In [20]:
###### .std_ is deprecated, use .scale_

simple_pipe.named_steps['scaler'].scale_

array([7.79554011e+00, 8.46820631e+00, 1.92546121e+01, 8.21341117e+00,
       4.30581401e-01, 4.33717586e-01, 4.32871171e-01, 4.34835848e-01,
       1.68917583e-01, 1.68917583e-01, 4.65211583e-01, 4.65211583e-01,
       4.72968433e-01, 4.38792288e-01, 2.67245683e-01, 1.10667959e-02,
       3.48510628e-01, 3.49891384e-01, 3.50390844e-01, 3.50515490e-01,
       3.48636592e-01, 3.51261545e-01, 3.50266110e-01, 2.70700637e-01,
       2.75954044e-01, 2.76139029e-01, 2.73532858e-01, 2.79801796e-01,
       2.77612529e-01, 2.76139029e-01, 2.78527758e-01, 2.74094313e-01,
       2.79074809e-01, 2.79074809e-01, 2.75768880e-01, 2.02566549e-01,
       1.98368429e-01, 2.01178774e-01, 1.96085646e-01, 1.98934278e-01,
       2.00620453e-01, 1.99779482e-01, 1.98934278e-01, 2.00899845e-01,
       1.98934278e-01, 2.00340597e-01, 1.97516047e-01, 1.98651593e-01,
       1.96945362e-01, 1.98368429e-01, 2.01178774e-01, 2.00340597e-01,
       2.03118487e-01, 2.04490531e-01, 2.00060274e-01, 1.99498220e-01,
      

In [21]:
###### Display $\beta_i$ for Lasso Model

simple_pipe.named_steps['lasso'].coef_

array([ 37.81956127,  15.07856264, -20.97367854,  -4.80325321,
        -8.67445325,   0.        ,  -0.        ,  16.55810977,
         0.        ,  -0.        ,  -0.        ,   0.        ,
         1.69301497,  -0.        , -13.9527773 ,  -0.16920321,
        -0.97024358,  -0.45100847,   0.        ,   0.        ,
         0.24940573,   1.67373388,  -3.22686251,  -3.55528056,
        -0.        ,   0.        ,  -0.        ,   5.86413596,
         0.        ,  -7.73988209,  -3.77614438,   3.19136327,
         1.1122235 ,   0.        ,   0.        , -27.62730192,
       -30.25156015, -31.97763363, -33.90388915, -33.8726006 ,
       -31.07937045, -18.90032333,   4.83038122,  34.76910242,
         3.98765501,  -6.1951688 ,  -1.00767564,   3.73829618,
         3.83990337,   0.        ,   1.67571803,  13.79605162,
        44.87442287,  41.18548178,  19.23742649,   2.42356056,
        -6.19354438, -12.34345661, -20.45458108])

####  The `make_pipeline`  helper function

While `Pipeline` gives us the ability to explicitly name our steps, this can be cumbersome, especially when we may not care what are steps are named. If this is the case, we use `make_pipeline`.

Let's execute the same pipeline, except this time we'll use `make_pipeline`.

In [22]:
###### Import `make_pipeline` helper function 

from sklearn.pipeline import make_pipeline

In [23]:
###### Define a second `Pipeline` using `make_pipeline`

another_pipe = make_pipeline(
    StandardScaler(),
    Lasso()
)

In [24]:
###### Fit second `Pipeline`

another_pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lasso', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))])

In [25]:
###### Display training score

another_pipe.score(X_train, y_train)

0.6388499611022919

In [26]:
###### Display testing score

another_pipe.score(X_test, y_test)

0.6274260376321519

Even though we don't name them, `make_pipeline` still has a `.named_steps` attribute. It automatically assigns names to each step and we can access them similarly to how we did before.

In [27]:
###### Display named steps for `Pipeline`

another_pipe.named_steps

{'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'lasso': Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False)}

In [28]:
###### Display steps for `Pipeline`

another_pipe.steps

[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('lasso', Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False))]

#### Aside: Transformation pipelines

Although it's standard to have a pipeline end in a model, it's also possible to have a pipeline just for transformers, as shown below:

In [29]:
###### Import transformers from Scikit-Learn 

from sklearn.feature_selection import (SelectFromModel, 
                                       SelectKBest, 
                                       f_regression)

In [30]:
###### Make transformation pipeline

transformer_pipe = make_pipeline(
    SelectKBest(score_func=f_regression, k=40),
    StandardScaler(),
    SelectFromModel(Lasso())
)

In [31]:
###### Fit transformation pipeline

transformer_pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('selectkbest', SelectKBest(k=40, score_func=<function f_regression at 0x1167a8158>)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectfrommodel', SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
        norm_order=1, prefit=False, threshold=None))])

In [32]:
###### Use transformation pipeline to transform data

features_skb_scaled_sfm = transformer_pipe.transform(X_train)

In [33]:
###### Show shape of training set

X_train.shape

(8164, 59)

In [34]:
###### Show shape of transformed data

features_skb_scaled_sfm.shape

(8164, 36)

In [35]:
###### Show steps in transformation pipeline

transformer_pipe.steps

[('selectkbest',
  SelectKBest(k=40, score_func=<function f_regression at 0x1167a8158>)),
 ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('selectfrommodel',
  SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False),
          norm_order=1, prefit=False, threshold=None))]

In [36]:
from sklearn.preprocessing import FunctionTransformer

In [37]:
###### Make transformation pipeline

transformer_pipe = make_pipeline(
    SelectKBest(score_func=f_regression, k=40),
    FunctionTransformer(lambda x: x + 1),
    FunctionTransformer(np.log),    
    StandardScaler(),
    SelectFromModel(Lasso())
)

In [38]:
transformer_pipe.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('selectkbest', SelectKBest(k=40, score_func=<function f_regression at 0x1167a8158>)), ('functiontransformer-1', FunctionTransformer(accept_sparse=False,
          func=<function <lambda> at 0x1167e0840>, inv_kw_args=None,
          inverse_func=None, kw_args=None, pass_y='deprecated',
      ...ction='cyclic', tol=0.0001, warm_start=False),
        norm_order=1, prefit=False, threshold=None))])

#### Using Pipelines with `GridSearchCV`

So far, we've only chained transformers and models together in a `pipeline`. What if we want to use GridSearch to tune our model in the `pipeline`?

Since we have to refer to our steps by name, let's use `Pipeline` instead of `make_pipeline`. 

Let's make a pipeline with the following steps:

    ('skb', SelectKBest(score_func=f_regression, k=40)),
    ('scaler', StandardScaler()),
    ('sfm', SelectFromModel(Lasso())),
    ('regr', ElasticNet())

In [39]:
###### Load necessary models

from sklearn.model_selection import (GridSearchCV, 
                                     ShuffleSplit)
from sklearn.linear_model import ElasticNet

In [40]:
###### Create model pipeline

pipe_for_gs = Pipeline([
    ('skb', SelectKBest(score_func=f_regression, k=40)),
    ('scaler', StandardScaler()),
    ('sfm', SelectFromModel(Lasso())),
    ('regr', ElasticNet())
])

In [41]:
pipe_for_gs.steps

[('skb', SelectKBest(k=40, score_func=<function f_regression at 0x1167a8158>)),
 ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)),
 ('sfm',
  SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
     normalize=False, positive=False, precompute=False, random_state=None,
     selection='cyclic', tol=0.0001, warm_start=False),
          norm_order=1, prefit=False, threshold=None)),
 ('regr', ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
        max_iter=1000, normalize=False, positive=False, precompute=False,
        random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]

Next, let's make our parameter grid. When using a `Pipeline`, we need to specify which step our params are for. To do that, we use the name we gave the step (in this case `'rf'` for `RandomForestRegressor`), with a **dunder** to reference a parameter for that model. 

As an example, if we wanted to tune `ElasticNet`'s `l1_ratio` parameter, we use `regr__l1_ratio:[.1,.5,.9]`. 

Let's fill out the params below to tune `alpha` and `l1_ratio`:

In [42]:
###### Define parameter grid

params = {
    'regr__l1_ratio':[.1,.3,.5,.7,.9],
    'regr__alpha':np.logspace(-3,3,7)
}

Now pass your pipeline into `GridSearchCV` with your parameters, using `ShuffleSplit`

In [43]:
###### Create grid search model pipeline

gspipe = GridSearchCV(
    pipe_for_gs, 
    param_grid=params, 
    cv=ShuffleSplit(n_splits=5, random_state=42), 
    n_jobs=-1
)

In [44]:
###### Fit the grid search model pipeline

gspipe.fit(X_train, y_train.ravel())

GridSearchCV(cv=ShuffleSplit(n_splits=5, random_state=42, test_size='default',
       train_size=None),
       error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('skb', SelectKBest(k=40, score_func=<function f_regression at 0x1167a8158>)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('sfm', SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precomput...alse, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'regr__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9], 'regr__alpha': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [45]:
###### Display best score for grid search model pipeline

gspipe.best_score_

0.6441703040776783

To get the `.steps` or `.named_steps`, we need to access `GridSearchCV`'s `.best_estimator_` parameter, which contains our `Pipeline`. How do we access our model? Our scaler?

In [46]:
###### Display named steps for best estimator

gspipe.best_estimator_.named_steps

{'skb': SelectKBest(k=40, score_func=<function f_regression at 0x1167a8158>),
 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'sfm': SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False),
         norm_order=1, prefit=False, threshold=None),
 'regr': ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.9,
       max_iter=1000, normalize=False, positive=False, precompute=False,
       random_state=None, selection='cyclic', tol=0.0001, warm_start=False)}

In [47]:
###### Display regression model

best_enet = gspipe.best_estimator_.named_steps['regr']

In [48]:
best_enet.coef_

array([ 39.92653743,  10.10050404, -19.59674176,  -6.9950194 ,
        -7.23595587,  17.10287993,   2.74715023, -14.92393049,
        -5.73295031,  -1.57377095,   7.71882526,   2.13610853,
        -7.04904951,  -3.25638783,   5.10841142,   2.36382364,
       -26.11055493, -28.89860596, -30.62160723, -32.66509316,
       -32.59217767, -29.80127324, -17.64190498,  38.11542642,
         7.53283836,   7.71971398,   7.85236345,   4.14529761,
         5.98572129,  18.02889019,  49.07576023,  45.28690045,
        23.15600355,   6.06896904, -10.67302863, -18.89872011])

In [49]:
###### Display `SelectFromModel` transformer

gspipe.best_estimator_.named_steps['sfm']

SelectFromModel(estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
        norm_order=1, prefit=False, threshold=None)